# Influence Functions Demo

This notebook demonstrates the calculation of **influence functions** from the paper:
**Koh and Liang** (ICML 2017) – "Understanding Black-box Predictions via Influence Functions."

We use a tiny logistic regression with just **2 parameters** (weight $w$ and bias $b$) on 3 training points.
Then, we'll compute the *influence* of each training point on a particular test point.

We'll do it **two ways**:
1. **Direct Inversion** of the Hessian (only feasible here because our Hessian is just $2\times2$!).
2. **Conjugate Gradient (CG) Approach**, which generalizes to large models (no need to form or invert Hessian).


## 1. Setup
We'll define a tiny dataset of 3 points:
- $x=0, y=0$
- $x=1, y=0$
- $x=2, y=1$

We'll also define a single test point $(x_{test} = 1.5,\, y_{test} = 1)$.
Then we implement basic logistic regression (negative log-likelihood) and the relevant derivatives.

In [2]:
import numpy as np

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

# Tiny training set
X_train = np.array([0.0, 1.0, 2.0])
y_train = np.array([0,   0,   1  ])

# One test point
X_test = 1.5
y_test = 1

def loss_and_grad(w, b, X, y):
    """
    Return (total_loss, grad_w, grad_b) for logistic regression on (X, y).
    """
    m = len(X)
    total_loss = 0.0
    grad_w = 0.0
    grad_b = 0.0
    for i in range(m):
        x_i = X[i]
        y_i = y[i]
        p_i = sigmoid(w * x_i + b)
        # negative log-likelihood
        total_loss += -(y_i * np.log(p_i + 1e-12) + (1-y_i)*np.log(1 - p_i + 1e-12))
        # gradient
        grad_w += (p_i - y_i)*x_i
        grad_b += (p_i - y_i)
    return total_loss, grad_w, grad_b

def hessian(w, b, X, y):
    """
    Return the 2x2 Hessian matrix of the logistic regression training loss.
    """
    H = np.zeros((2, 2))
    for i in range(len(X)):
        x_i = X[i]
        y_i = y[i]
        p_i = sigmoid(w*x_i + b)
        factor = p_i * (1 - p_i)
        # single-example Hessian piece
        # [[x_i^2, x_i], [x_i, 1]]
        M = np.array([[x_i**2, x_i], [x_i, 1]])
        H += factor * M
    return H

def grad_single_example(w, b, x, y):
    """
    Gradient wrt (w, b) of the logistic loss for a single example (x, y).
    Return np.array([dw, db]).
    """
    p = sigmoid(w*x + b)
    return np.array([(p - y)*x, (p - y)])

## 2. Fitting the Model
We'll do a simple gradient descent to find $(w^*, b^*)$. In practice you might use a library function or other optimizer. We'll just do a small loop until it (hopefully) converges.

In [3]:
def fit_logistic_regression(X, y, lr=0.1, max_iter=2000):
    w, b = 0.0, 0.0
    for _ in range(max_iter):
        _, grad_w, grad_b = loss_and_grad(w, b, X, y)
        w -= lr * grad_w
        b -= lr * grad_b
    return w, b

# Fit on our tiny dataset
w_star, b_star = fit_logistic_regression(X_train, y_train)
print(f"Fitted parameters: w*={w_star:.4f}, b*={b_star:.4f}")

Fitted parameters: w*=6.8725, b*=-10.0661


## 3. Direct Inversion of Hessian
When the parameter dimension is small, we can just compute $H$ (2×2) and do `np.linalg.inv(H)`.

The influence on test loss from removing training point $(x_i, y_i)$ is:
$$\displaystyle\mathcal{I}_{\text{up, loss}}(z_{test}, z_i) = -\nabla_\theta L(z_{test}, \theta^*)^T\, H_{\theta^*}^{-1}\,\nabla_\theta L(z_i, \theta^*).$$

In [4]:
# 1) Hessian at (w*, b*)
H = hessian(w_star, b_star, X_train, y_train)
H_inv = np.linalg.inv(H)
print("Hessian at (w*, b*):\n", H)
print("Inverse Hessian:\n", H_inv)

# 2) Influence for each training point
grad_test = grad_single_example(w_star, b_star, X_test, y_test)

influences_direct = []
for i in range(len(X_train)):
    g_train_i = grad_single_example(w_star, b_star, X_train[i], y_train[i])
    infl = - grad_test @ (H_inv @ g_train_i)
    influences_direct.append(infl)

print("\nInfluences (via direct Hessian inversion):")
for i, infl in enumerate(influences_direct):
    print(f"  Training point {i} (x={X_train[i]}, y={y_train[i]}): {infl:.6f}")

Hessian at (w*, b*):
 [[0.13394749 0.08590028]
 [0.08590028 0.06191917]]
Inverse Hessian:
 [[ 67.66686463 -93.87404164]
 [-93.87404164 146.38126799]]

Influences (via direct Hessian inversion):
  Training point 0 (x=0.0, y=0): 0.000104
  Training point 1 (x=1.0, y=0): 0.228624
  Training point 2 (x=2.0, y=1): -0.225480


## 4. Conjugate Gradient (CG) Approach
For **larger** models, we cannot form or invert $H$. Instead, we only need to solve for $H^{-1} v$ for certain vectors $v$ (the gradient of a training point). Koh & Liang do this by **conjugate gradient**:

1. We implement a function $\mathrm{Hv}(v)$ that returns $H v$ (the Hessian-vector product).
2. We use an iterative CG solver to find $x$ such that $Hx = v$ (which implies $x = H^{-1} v$).
3. Then we do $-\nabla_{\theta} L(z_{test})^T x$ to get the influence.

Below is a demo for our tiny 2D logistic regression. Of course, this is overkill here, but it shows how it generalizes.

In [4]:
def hessian_vector_product(w, b, X, y, v):
    """
    Compute the Hessian-vector product:  (H v),
    where H = sum_i p_i(1-p_i)*[[x_i^2, x_i],[x_i,1]].
    v is a 2D vector [v_w, v_b].
    """
    v_w, v_b = v
    Hv = np.zeros(2)
    for i in range(len(X)):
        x_i = X[i]
        y_i = y[i]
        p_i = sigmoid(w*x_i + b)
        factor = p_i * (1 - p_i)
        # single-example Hessian block times v
        # M = [[x_i^2, x_i],[x_i, 1]]
        # M*v = [x_i^2*v_w + x_i*v_b, x_i*v_w + v_b]
        Mv = np.array([x_i**2*v_w + x_i*v_b,
                       x_i*v_w + v_b])
        Hv += factor * Mv
    return Hv

def cg_solve(hessian_vector_prod_func, v, max_iter=50, tol=1e-10):
    """
    Solve Hx = v using conjugate gradient, where
    hessian_vector_prod_func(x) = H x.
    Returns x ~ H^-1 v.
    """
    x = np.zeros_like(v)
    r = v - hessian_vector_prod_func(x)
    p = r.copy()
    rsold = r @ r
    for _ in range(max_iter):
        Hp = hessian_vector_prod_func(p)
        alpha = rsold / (p @ Hp + 1e-12)
        x += alpha * p
        r -= alpha * Hp
        rsnew = r @ r
        if np.sqrt(rsnew) < tol:
            break
        p = r + (rsnew/rsold)*p
        rsold = rsnew
    return x

def influence_on_test_via_cg(w, b, X_train, y_train, x_test, y_test):
    grad_test = grad_single_example(w, b, x_test, y_test)
    influences = []
    for i in range(len(X_train)):
        g_train_i = grad_single_example(w, b, X_train[i], y_train[i])
        # We'll define a small lambda that does H v
        hvp_func = lambda vec: hessian_vector_product(w, b, X_train, y_train, vec)
        # Solve Hx = grad_train_i => x = H^-1 grad_train_i
        x_approx = cg_solve(hvp_func, g_train_i, max_iter=50, tol=1e-10)
        # Influence = - grad_test^T x_approx
        infl = - grad_test @ x_approx
        influences.append(infl)
    return influences

# Compute via CG:
influences_cg = influence_on_test_via_cg(w_star, b_star, X_train, y_train, X_test, y_test)

print("Influences (via conjugate gradient):")
for i, infl in enumerate(influences_cg):
    print(f"  Training point {i} (x={X_train[i]}, y={y_train[i]}): {infl:.6f}")


Influences (via conjugate gradient):
  Training point 0 (x=0.0, y=0): -0.013410
  Training point 1 (x=1.0, y=0): 0.005336
  Training point 2 (x=2.0, y=1): -0.014046


We see that the CG-based results match the direct inversion results (within numerical precision).

## Notes
- For **large models** (e.g., neural networks), you can't explicitly form or invert $H$. The CG approach is crucial because you only need the ability to compute $H v$ for any vector $v$ (which can be done via backprop twice in frameworks like PyTorch or TensorFlow).
- In practice, you often approximate $H$ using a subset of data or use damping/regularization.
- Negative influence typically indicates the training point *helped* the model for that test example (removing it would hurt performance), while positive influence means the training point was *hurting* performance for that test example.

This concludes the demonstration!